In [ ]:
! git clone https://github.com/niladridutt/DeepVCP-Pointcloud-Registration.git

Cloning into 'DeepVCP-Pointcloud-Registration'...
remote: Enumerating objects: 1333, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 1333 (delta 11), reused 19 (delta 7), pack-reused 1297
Receiving objects: 100% (1333/1333), 138.82 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (344/344), done.


In [ ]:
!wget https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip --no-check-certificate

--2023-04-15 10:12:35--  https://shapenet.cs.stanford.edu/media/modelnet40_normal_resampled.zip
Resolving shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)... 171.67.77.19
Connecting to shapenet.cs.stanford.edu (shapenet.cs.stanford.edu)|171.67.77.19|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 1705117335 (1.6G) [application/zip]
Saving to: ‘modelnet40_normal_resampled.zip’

modelnet40_normal_r 100%[===================>]   1.59G  58.1MB/s    in 29s     

2023-04-15 10:13:04 (56.1 MB/s) - ‘modelnet40_normal_resampled.zip’ saved [1705117335/1705117335]



In [ ]:
! unzip -q modelnet40_normal_resampled.zip

In [ ]:
!pip install --upgrade https://github.com/unlimblue/KNN_CUDA/releases/download/0.2/KNN_CUDA-0.2-py3-none-any.whl -q

In [ ]:
! pip install Ninja -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 4.7 MB/s eta 0:00:00


In [3]:
! pip install trimesh -q

In [1]:
import os
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scipy.spatial.transform import Rotation as R
import time
import pickle
import argparse
from utils import *

from deepVCP import DeepVCP
from ModelNet40Dataset import ModelNet40Dataset
from KITTIDataset import KITTIDataset
from CustomDataset import CustomDataset
from deepVCP_loss import deepVCP_loss
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

In [2]:
dataset = 'modelnet'
retrain_path = 'store'
model_path = 'final_model.pt'
full_dataset = True

In [7]:
num_epochs = 5
batch_size = 1
lr = 0.001
# loss balancing factor 
alpha = 0.5

print(f"Params: epochs: {num_epochs}, batch: {batch_size}, lr: {lr}, alpha: {alpha}\n")

# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")


Params: epochs: 5, batch: 1, lr: 0.001, alpha: 0.5

device: cuda


In [4]:
root = 'modelnet40_normal_resampled/'
shape_names = np.loadtxt(root+"modelnet10_shape_names.txt", dtype="str")
train_data= ModelNet40Dataset(root=root, augment=True, full_dataset=full_dataset, split='train')

# Total clouds 785


In [5]:
test_data = ModelNet40Dataset(root=root, augment=True, full_dataset=full_dataset,  split='test')

# Total clouds 197


In [8]:
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)


num_train = len(train_data)
num_test = len(test_data)
print('Train dataset size: ', num_train)
print('Test dataset size: ', num_test)

use_normal = dataset == "modelnet"

# Initialize the model
model = DeepVCP(use_normal=use_normal)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

Train dataset size:  785
Test dataset size:  197


DeepVCP(
  (FE1): feat_extraction_layer(
    (sa1): PointNetSetAbstraction(
      (mlp_convs): ModuleList(
        (0): Conv2d(6, 16, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
        (2): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (mlp_bns): ModuleList(
        (0-1): 2 x BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (sa2): PointNetSetAbstraction(
      (mlp_convs): ModuleList(
        (0): Conv2d(6, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (mlp_bns): ModuleList(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (sa3): PointNetSetAbstraction(

In [9]:
optim = Adam(model.parameters(), lr=lr)

# begin train 
model.train()
loss_epoch_avg = []
train_loss = []
for epoch in range(num_epochs):
    print(f"epoch #{epoch}")
    loss_epoch = []
    running_loss = 0.0

    for n_batch, (src, target, R_gt, t_gt, ) in enumerate(train_loader):
        start_time = time.time()
        # mini batch
        src, target, R_gt, t_gt = src.to(device), target.to(device), R_gt.to(device), t_gt.to(device)
        t_init = torch.randn((1, 3))
        R_prior = R_gt.clone()
        corruption = torch.FloatTensor(batch_size, 3, 3).uniform_(0.9, 1.1).cuda()
        R_prior = R_prior*corruption
        src_keypts, target_vcp = model(src, target, R_prior, t_init)
        # print('src_keypts shape', src_keypts.shape)
        # print('target_vcp shape', target_vcp.shape)
        # zero gradient 
        optim.zero_grad()
        loss, R_pred, t_pred = deepVCP_loss(src_keypts, target_vcp, R_gt, t_gt, alpha=0.5)

        # error metric for rigid body transformation
        r_pred = R.from_matrix(R_pred.squeeze(0).cpu().detach().numpy())
        r_pred_arr = torch.tensor(r_pred.as_euler('xyz', degrees=True)).reshape(1, 3)
        r_gt = R.from_matrix(R_gt.squeeze(0).cpu().detach().numpy())
        r_gt_arr = torch.tensor(r_gt.as_euler('xyz', degrees=True)).reshape(1, 3)
        pdist = nn.PairwiseDistance(p = 2)
        t_pred = t_pred.squeeze(-1)
        t_gt = t_gt.squeeze(-1)

        print("rotation error: ", pdist(r_pred_arr, r_gt_arr).mean())
        print("translation error: ", pdist(t_pred, t_gt).mean())

        # backward pass
        loss.backward()
        # update parameters 
        optim.step()

        running_loss += loss.item()
        loss_epoch += [loss.item()]
        print("--- %s seconds ---" % (time.time() - start_time))
        if (n_batch + 1) % 5 == 0:
            print("Epoch: [{}/{}], Batch: {}, Loss: {}".format(
                epoch, num_epochs, n_batch, loss.item()))
            running_loss = 0.0

    torch.save(model.state_dict(), "epoch_" + str(epoch) + "_model.pt")
    loss_epoch_avg += [sum(loss_epoch) / len(loss_epoch)]
    train_loss.append(sum(loss_epoch))
    print("loss epoch", loss_epoch)
    with open("training_loss_" + str(epoch) + ".txt", "wb") as fp:   #Pickling
        pickle.dump(loss_epoch, fp)


epoch #0
Processing file: bed_0001.txt
feature extraction time:  10.495105981826782
src_keypts_idx_unsqueezed:  torch.Size([1, 6, 64])
src_keypts:  torch.Size([1, 64, 6])
Grouping keypoints time:  0.02200007438659668
B:  1
K_topk:  64
nsample:  32
num_feat:  32
get_cat_feat_src time:  0.0


c:\Users\niladutt\.conda\envs\nsd\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tgt_pts_xyz:  torch.Size([1, 10000, 3])
ref_pts:  torch.Size([1, 10000, 3])
dist_normalize:  torch.Size([1, 32, 13824])
feat_weight_map:  torch.Size([1, 32, 32, 13824])
idx_1_mask:  tensor([[0]])
idx_1_mask_flatten:  tensor([0])
idx_2_mask:  tensor([  88, 9469, 6891,  ..., 9859, 4812, 8174], device='cuda:0')
get_cat_feat_tgt time:  0.11301946640014648
Loss: 0.331563321438198
rotation error:  tensor(2.2912, dtype=torch.float64)
translation error:  tensor(0.8204, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
--- 20.604880809783936 seconds ---
Processing file: bed_0002.txt


KeyboardInterrupt: 

In [ ]:
# save
print("Finished Training")
torch.save(model.state_dict(), model_path)